In [18]:
# Installations

%pip install --upgrade pip -q
%pip install mypy -q

%pip install numpy matplotlib pandas scipy -q
# %pip install setuptools wandb -q

%pip install openai python-dotenv -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter

king_lear_text = TextFileLoader("./data/KingLear.txt")
documents = king_lear_text.load_documents()

splitter = CharacterTextSplitter()
chunks = splitter.split_texts(documents)

In [20]:
from aimakerspace.vectordatabase import VectorDatabase

database = await VectorDatabase().abuild_from_list(chunks)

In [21]:
from dataclasses import dataclass
from enum import Enum

from openai import OpenAI
from openai.types.chat import (
    ChatCompletionMessage,
    ChatCompletionSystemMessageParam,
    ChatCompletionUserMessageParam,
    ChatCompletionAssistantMessageParam,
    ChatCompletionToolMessageParam,
    ChatCompletionFunctionMessageParam,
  )

from openai.types.chat.completion_create_params import ResponseFormat
from openai._types import NotGiven

from dotenv import dotenv_values

# WANDB = False

# if WANDB:
#   from wandb.integration.openai import autolog
#   autolog({"project": "First RAG App"})

ai_client = OpenAI(api_key=dotenv_values("../.env")["OPENAI_API_KEY"])

RawConversationMessage = ChatCompletionSystemMessageParam | ChatCompletionUserMessageParam | ChatCompletionAssistantMessageParam | ChatCompletionToolMessageParam | ChatCompletionFunctionMessageParam

class Role(Enum):
  System = "system"
  User = "user"
  Assistant = "assistant"
  Tool = "tool"

@dataclass(frozen=True, order=True, slots=True)
class Message():
  role: Role
  content: str

  def into(self) -> RawConversationMessage:
    return {"role": self.role.value, "content": self.content} # type:ignore

  @classmethod
  def from_raw(cls, raw: ChatCompletionMessage) -> "Message":
    return cls(Role(raw.role), raw.content or "")

def system(message: str) -> Message:
  return Message(Role.System, message)

def user(message: str) -> Message:
  return Message(Role.User, message)

def respond(messages: list[Message], response_format: ResponseFormat | NotGiven=NotGiven()) -> Message:
  raw_messages = [message.into() for message in messages]

  return Message.from_raw(
    ai_client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=raw_messages,
      response_format=response_format
    ).choices[0].message
  )

from dataclasses import field

@dataclass(slots=True)
class Chatbot:
    messages: list[Message] = field(default_factory=list)

    def respond(self, response_format: ResponseFormat | NotGiven=NotGiven()) -> Message:
        response = respond(self.messages, response_format = response_format)
        self.messages.append(response)
        return response

    def converse(self, chat: Message, response_format: ResponseFormat | NotGiven=NotGiven()) -> Message:
        self.messages.append(chat)
        return self.respond(response_format=response_format)

In [23]:
king_lear_expert = Chatbot(
    [
        system(
"""
You will be given context and a query. Use the context to answer the query.
If you do not know the answer or there's no provided context, respond with "I don't know".
"""
)
    ]
)

question = "Who is King Lear's enemy?"

matching_chunks = database.search_by_text(question, k=4)
context = "\n".join((context[0] for context in matching_chunks))

response = king_lear_expert.converse(
    user(
f"""
Context:
{context}

Query:
{question}
"""
    )
).content

print(response)

King Lear's enemies are his daughters, Goneril and Regan, who turn against him during the course of the play.
